<a href="https://colab.research.google.com/github/data602sps/assignments/blob/master/05_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 7**

# **Weeks 8 & 9 - Pandas**
* In this homework assignment, you will explore and analyze a public dataset of your choosing. Since this assignment is “open-ended” in nature, you are free to expand upon the requirements below. However, you must meet the minimum requirments as indicated in each section. 

* You must use Pandas as the **primary tool** to process your data.

* The preferred method for this analysis is in a .ipynb file. Feel free to use whichever platform of your choosing.  
 * https://www.youtube.com/watch?v=inN8seMm7UI (Getting started with Colab).

* Your data should need some "work", or be considered "dirty".  You must show your skills in data cleaning/wrangling.

### **Some data examples:**
•	https://www.data.gov/

•	https://opendata.cityofnewyork.us/

•	https://datasetsearch.research.google.com/

•	https://archive.ics.uci.edu/ml/index.php

### **Resources:**

•	https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html 

•	https://pandas.pydata.org/pandas-docs/stable/user_guide/visualization.html


### **Headings or comments**
**You are required to make use of comments, or headings for each section.  You must explain what your code is doing, and the results of running your code.**  Act as if you were giving this assignment to your manager - you must include clear and descriptive information for each section.

### **You may work as a group or indivdually on this assignment.**


# Introduction

In this section, please describe the dataset you are using.  Include a link to the source of this data.  You should also provide some explanation on why you choose this dataset.

The data that I am using is retail data from across the UK. The clients are ordering from an online retail store which primarily sells giftware. The cost of items, the amount that were ordered, the country of origin, and the order ID are all available in the dataset. The dataset can be found at: https://archive.ics.uci.edu/dataset/502/online+retail+ii

I chose this dataset because it felt like something that an analyst might look at to determine important metrics, such as best selling products or important clients. Things that I hope to determine in this dataset include: biggest spenders, who is buying the most expensive products, and who has the biggest orders. 

______________
# Data Exploration
Import your dataset into your .ipynb, create dataframes, and explore your data.  

Include: 

* Summary statistics means, medians, quartiles, 
* Missing value information
* Any other relevant information about the dataset.  



## Import Packages:

In [222]:
import pandas as pd
import numpy as np
import seaborn
import datetime

## Create Dataframes:

In [223]:
df1 = pd.read_excel("https://github.com/sphill12/data_602/raw/main/online_retail_II.xlsx",sheet_name= "Year 2009-2010")
df2 = pd.read_excel("https://github.com/sphill12/data_602/raw/main/online_retail_II.xlsx",sheet_name= "Year 2010-2011")

In [224]:
df1["year"] = "2009-2010"
df2["year"] = "2010-2011"
df = pd.concat([df1,df2])

## Look at general dataframe information:

In [225]:
df.shape

(1067371, 9)

In [226]:
df.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
year                   object
dtype: object

## What countries are included

In [227]:
print(pd.unique(df["Country"]))

['United Kingdom' 'France' 'USA' 'Belgium' 'Australia' 'EIRE' 'Germany'
 'Portugal' 'Japan' 'Denmark' 'Nigeria' 'Netherlands' 'Poland' 'Spain'
 'Channel Islands' 'Italy' 'Cyprus' 'Greece' 'Norway' 'Austria' 'Sweden'
 'United Arab Emirates' 'Finland' 'Switzerland' 'Unspecified' 'Malta'
 'Bahrain' 'RSA' 'Bermuda' 'Hong Kong' 'Singapore' 'Thailand' 'Israel'
 'Lithuania' 'West Indies' 'Lebanon' 'Korea' 'Brazil' 'Canada' 'Iceland'
 'Saudi Arabia' 'Czech Republic' 'European Community']


## Are there Missing Values?

In [228]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
year                0
dtype: int64

## The dataframe only contains missing values in the description and the customer ID. There still could be non missing values that need to be removed/corrected as well

## Retrieve Summary information for relevant columns:

In [229]:
df[["Quantity","Price"]].describe().round()

,Quantity,Price
count,1067371.0,1067371.0
mean,10.0,5.0
std,173.0,124.0
min,-80995.0,-53594.0
25%,1.0,1.0
50%,3.0,2.0
75%,10.0,4.0
max,80995.0,38970.0


## The minimum value and maximum value for quantity are suspicious. We can isolate these to look at what is going on

In [230]:
df[df["Quantity"]== 80995]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,year
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,2010-2011


In [231]:
df[df["Quantity"] == -80995]

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,year
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,2010-2011


## Based on this, we can assume that the order was an accident. It might be best to remove this value later for the sake of analyis. We can make a list of these suspicious rows to drop later

In [232]:
## Look and see if any other values are suspicious
df.sort_values(by = ["Quantity"]).head(10)


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,year
540422,C581484,23843,"PAPER CRAFT , LITTLE BIRDIE",-80995,2011-12-09 09:27:00,2.08,16446.0,United Kingdom,2010-2011
61624,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,-74215,2011-01-18 10:17:00,1.04,12346.0,United Kingdom,2010-2011
303996,519017,22759,NaN,-9600,2010-08-13 09:14:00,0.00,NaN,United Kingdom,2009-2010
225529,556690,23005,printing smudges/thrown away,-9600,2011-06-14 10:37:00,0.00,NaN,United Kingdom,2010-2011
225530,556691,23005,printing smudges/thrown away,-9600,2011-06-14 10:37:00,0.00,NaN,United Kingdom,2010-2011
4268,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom,2010-2011
507225,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02 14:23:00,0.03,15838.0,United Kingdom,2009-2010
156488,504311,22197,NaN,-9200,2010-04-12 14:39:00,0.00,NaN,United Kingdom,2009-2010
225528,556687,23003,Printing smudges/thrown away,-9058,2011-06-14 10:36:00,0.00,NaN,United Kingdom,2010-2011
428975,530348,16235,?,-9000,2010-11-02 15:48:00,0.00,NaN,United Kingdom,2009-2010


In [233]:
drop_list = [540421, 540422, 61624, 61619]

In [234]:
df.sort_values(by = ["Quantity"], ascending= False).head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,year
540421,581483,23843,"PAPER CRAFT , LITTLE BIRDIE",80995,2011-12-09 09:15:00,2.08,16446.0,United Kingdom,2010-2011
61619,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,74215,2011-01-18 10:01:00,1.04,12346.0,United Kingdom,2010-2011
90857,497946,37410,BLACK AND WHITE PAISLEY FLOWER MUG,19152,2010-02-15 11:57:00,0.10,13902.0,Denmark,2009-2010
127168,501534,21091,SET/6 WOODLAND PAPER PLATES,12960,2010-03-17 13:09:00,0.10,13902.0,Denmark,2009-2010
127166,501534,21099,SET/6 STRAWBERRY PAPER CUPS,12960,2010-03-17 13:09:00,0.10,13902.0,Denmark,2009-2010
127169,501534,21085,SET/6 WOODLAND PAPER CUPS,12744,2010-03-17 13:09:00,0.10,13902.0,Denmark,2009-2010
502122,578841,84826,ASSTD DESIGN 3D PAPER STICKERS,12540,2011-11-25 15:57:00,0.00,13256.0,United Kingdom,2010-2011
127167,501534,21092,SET/6 STRAWBERRY PAPER PLATES,12480,2010-03-17 13:09:00,0.10,13902.0,Denmark,2009-2010
192197,507637,84016,FLAG OF ST GEORGE CAR FLAG,10200,2010-05-10 14:55:00,0.00,NaN,United Kingdom,2009-2010
135028,502269,21982,PACK OF 12 SUKI TISSUES,10000,2010-03-23 15:36:00,0.25,17940.0,United Kingdom,2009-2010


## From looking at the highest and lowest quantity values we can see some interesting data entries. Some entries are not with clients, such as the "printing smudges/thrown away" rows. It might be best to remove these, as they are not very informative. We can also see some rows that contain NaN values for the description, or "?". There is a decent amount of cleaning that potentially needs to be done.

# Data Wrangling
Create a subset of your original data and perform the following.  

1. Modify multiple column names.

2. Look at the structure of your data – are any variables improperly coded? Such as strings or characters? Convert to correct structure if needed.

3. Fix missing and invalid values in data.

4. Create new columns based on existing columns or calculations.

5. Drop column(s) from your dataset.

6. Drop a row(s) from your dataset.

7. Sort your data based on multiple variables. 

8. Filter your data based on some condition. 

9. Convert all the string values to upper or lower cases in one column.

10. Check whether numeric values are present in a given column of your dataframe.

11. Group your dataset by one column, and get the mean, min, and max values by group. 
  * Groupby()
  * agg() or .apply()

12. Group your dataset by two columns and then sort the aggregated results within the groups. 

**You are free (and should) to add on to these questions.  Please clearly indicate in your assignment your answers to these questions.**

# 1.)

## I prefer my column names to be lowercase, with underscores instead of spaces. I will change the columns to this format

In [235]:
column_names = [name.lower() for name in df.columns]

column_names[4] = "invoice_date"
column_names[-3] = "customer_id"
column_names[1] = "stock_code"
df.columns = column_names
df.columns

Index(['invoice', 'stock_code', 'description', 'quantity', 'invoice_date',
       'price', 'customer_id', 'country', 'year'],
      dtype='object')

# 2.)
## Look at the structure of data:

In [236]:
print("The data frame is shaped like:" + str(df.shape))
print("The variable types are:" +"\n" +  str(df.dtypes))

The data frame is shaped like:(1067371, 9)
The variable types are:
invoice                 object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
price                  float64
customer_id            float64
country                 object
year                    object
dtype: object


## All of the columns are the appropriate type. Since we don't need the hourly invoice data, we can convert it to be just the date:

In [237]:
df["invoice_date"] = df["invoice_date"].apply(lambda x : x.strftime("%Y-%m-%d"))

In [238]:
df.head()

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,year
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom,2009-2010
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,2009-2010
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,2009-2010
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01,2.10,13085.0,United Kingdom,2009-2010
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01,1.25,13085.0,United Kingdom,2009-2010


# 3.)

## Because there is already so much data available, and the columns that contain NA's are those without clients, I will simply drop all of them

In [239]:
df =df.dropna()
df.head()

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,year
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom,2009-2010
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,2009-2010
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,2009-2010
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01,2.10,13085.0,United Kingdom,2009-2010
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01,1.25,13085.0,United Kingdom,2009-2010


In [240]:
df.isnull().sum()

invoice         0
stock_code      0
description     0
quantity        0
invoice_date    0
price           0
customer_id     0
country         0
year            0
dtype: int64

# 4.)

## We can assume that the prices listed are unit prices, and we can create a total cost column:

In [241]:
df["total_cost"] = df["quantity"] * df["price"]
df.head()

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,year,total_cost
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom,2009-2010,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,2009-2010,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,2009-2010,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01,2.10,13085.0,United Kingdom,2009-2010,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01,1.25,13085.0,United Kingdom,2009-2010,30.0


# 5.)

## There aren't any columns that I would remove but for demonstration we can get rid of the year column

In [242]:
df.drop(["year"], axis = 1)

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,total_cost
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom,83.40
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,81.00
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,81.00
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01,2.10,13085.0,United Kingdom,100.80
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01,1.25,13085.0,United Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09,4.15,12680.0,France,16.60
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09,4.95,12680.0,France,14.85


# 6.)

## There are more rows that aren't representative of clients, such as the postage column. We can remove these, along with the spefic rows that were on the drop list:

In [243]:
df = df[df["stock_code"] != "POST"]
df = df.drop(labels= drop_list, axis = 0)
df = df[df["stock_code"] != "M"]
df = df[df["stock_code"] != "D"]
df = df[df["stock_code"] != "DOT"]
df = df[df["stock_code"] != "ADJUST"]
df = df[df["stock_code"]!= "CRUK"]
print(drop_list)

[540421, 540422, 61624, 61619]


In [244]:
df.sort_values(by = ["quantity"], ascending = False)

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,year,total_cost
90857,497946,37410,BLACK AND WHITE PAISLEY FLOWER MUG,19152,2010-02-15,0.10,13902.0,Denmark,2009-2010,1915.20
127168,501534,21091,SET/6 WOODLAND PAPER PLATES,12960,2010-03-17,0.10,13902.0,Denmark,2009-2010,1296.00
127166,501534,21099,SET/6 STRAWBERRY PAPER CUPS,12960,2010-03-17,0.10,13902.0,Denmark,2009-2010,1296.00
127169,501534,21085,SET/6 WOODLAND PAPER CUPS,12744,2010-03-17,0.10,13902.0,Denmark,2009-2010,1274.40
502122,578841,84826,ASSTD DESIGN 3D PAPER STICKERS,12540,2011-11-25,0.00,13256.0,United Kingdom,2010-2011,0.00
...,...,...,...,...,...,...,...,...,...,...
359630,C524235,16047,POP ART PEN CASE & PENS,-5184,2010-09-28,0.08,14277.0,France,2009-2010,-414.72
359670,C524235,21096,SET/6 FRUIT SALAD PAPER PLATES,-7008,2010-09-28,0.13,14277.0,France,2009-2010,-911.04
359669,C524235,21088,SET/6 FRUIT SALAD PAPER CUPS,-7128,2010-09-28,0.08,14277.0,France,2009-2010,-570.24
507225,C536757,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,-9360,2010-12-02,0.03,15838.0,United Kingdom,2009-2010,-280.80


# 8.)
## We can filter for multiple qualities. Lets look at the orders from 2009 in the United Kingdom, that have a price over 1000 dollars:

In [245]:
df[(df["country"] == "United Kingdom") & (df["total_cost"] >= 1000 ) & (df["year"] == "2009-2010")]

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,year,total_cost
575,489523,84879,ASSORTED COLOUR BIRD ORNAMENT,800,2009-12-01,1.45,12931.0,United Kingdom,2009-2010,1160.00
3290,489675,15056BL,EDWARDIAN PARASOL BLACK,240,2009-12-02,4.60,13777.0,United Kingdom,2009-2010,1104.00
7007,490010,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,480,2009-12-03,2.10,15061.0,United Kingdom,2009-2010,1008.00
7302,490018,21981,PACK OF 12 WOODLAND TISSUES,4320,2009-12-03,0.25,17940.0,United Kingdom,2009-2010,1080.00
7303,490018,21967,PACK OF 12 SKULL TISSUES,5184,2009-12-03,0.25,17940.0,United Kingdom,2009-2010,1296.00
...,...,...,...,...,...,...,...,...,...,...
519374,537659,22188,BLACK HEART CARD HOLDER,1008,2010-12-07,2.31,18102.0,United Kingdom,2009-2010,2328.48
519376,537659,21623,VINTAGE UNION JACK MEMOBOARD,600,2010-12-07,6.38,18102.0,United Kingdom,2009-2010,3828.00
519378,537659,82484,WOOD BLACK BOARD ANT WHITE FINISH,600,2010-12-07,4.78,18102.0,United Kingdom,2009-2010,2868.00
519379,537659,22833,HALL CABINET WITH 3 DRAWERS,72,2010-12-07,32.69,18102.0,United Kingdom,2009-2010,2353.68


# 7.)

## Now lets sort in descending order by the total cost, and most recent date:

In [246]:
df[(df["country"] == "United Kingdom") & (df["total_cost"] >= 1000 ) & (df["year"] == "2009-2010")].sort_values(by = ["total_cost", "invoice_date"], ascending= [False,False])

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,year,total_cost
432176,530715,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,9360,2010-11-04,1.69,15838.0,United Kingdom,2009-2010,15818.40
228042,511465,15044A,PINK PAPER PARASOL,3500,2010-06-08,2.55,18008.0,United Kingdom,2009-2010,8925.00
379875,525968,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,3120,2010-10-08,1.66,15838.0,United Kingdom,2009-2010,5179.20
358821,524181,21622,VINTAGE UNION JACK CUSHION COVER,648,2010-09-27,6.89,17450.0,United Kingdom,2009-2010,4464.72
452200,532358,84879,ASSORTED COLOUR BIRD ORNAMENT,2880,2010-11-11,1.45,12931.0,United Kingdom,2009-2010,4176.00
...,...,...,...,...,...,...,...,...,...,...
7304,490018,21984,PACK OF 12 PINK PAISLEY TISSUES,4008,2009-12-03,0.25,17940.0,United Kingdom,2009-2010,1002.00
7305,490018,21980,PACK OF 12 RED SPOTTY TISSUES,4008,2009-12-03,0.25,17940.0,United Kingdom,2009-2010,1002.00
203866,509037,62018,SOMBRERO,800,2010-05-20,1.25,13953.0,United Kingdom,2009-2010,1000.00
53228,494243,84830,LARGE ENGLISH ARBORETUM,100,2010-01-12,10.00,18102.0,United Kingdom,2009-2010,1000.00


# 9.)

In [247]:
df["description"] = df["description"].apply(lambda x: x.upper())
df.head()

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,year,total_cost
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom,2009-2010,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,2009-2010,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,2009-2010,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01,2.10,13085.0,United Kingdom,2009-2010,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01,1.25,13085.0,United Kingdom,2009-2010,30.0


# 10.)

In [248]:
pd.api.types.is_numeric_dtype(df["price"])
pd.api.types.is_numeric_dtype(df["total_cost"])

True

## We can see that the price column is numeric

# 11.)

In [249]:
df.head()

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,year,total_cost
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01,6.95,13085.0,United Kingdom,2009-2010,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,2009-2010,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01,6.75,13085.0,United Kingdom,2009-2010,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01,2.10,13085.0,United Kingdom,2009-2010,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01,1.25,13085.0,United Kingdom,2009-2010,30.0


In [250]:
pd.DataFrame(df.groupby(["country"])["total_cost"].mean()).sort_values(by = "total_cost", ascending= False)

,total_cost
country,
Netherlands,108.754665
Australia,87.692647
Denmark,81.430669
Japan,75.884516
Sweden,65.400521
Thailand,40.401842
Singapore,39.633012
Lebanon,37.641778
EIRE,36.049161


## The Netherlands has the highest average cost for their orders

In [251]:
pd.DataFrame(df.groupby(["country"])["total_cost"].min()).sort_values(by = "total_cost")

,total_cost
country,
United Kingdom,-6539.40
Denmark,-3630.80
France,-1653.12
Japan,-1591.20
EIRE,-1224.00
Sweden,-1188.00
Spain,-950.40
Channel Islands,-885.00
Netherlands,-495.00


## The United Kingdom has the smallest order

In [252]:
pd.DataFrame(df[["country","total_cost"]].groupby(["country"]).max()).sort_values(by = "total_cost", ascending=False)

,total_cost
country,
United Kingdom,38970.00
Netherlands,4992.00
Japan,3794.40
Denmark,3630.80
EIRE,2365.20
Australia,1718.40
France,1653.12
Spain,1350.00
Sweden,1188.00


## The United Kingdom also has the largest order

# 12.)

In [253]:
grouped_df = df[["country", "description","quantity","total_cost"]].sort_values(by = "quantity",ascending=False).groupby(by = ["country","description"])
grouped_df = grouped_df.head()
grouped_df.head(20)

,country,description,quantity,total_cost
90857,Denmark,BLACK AND WHITE PAISLEY FLOWER MUG,19152,1915.20
127168,Denmark,SET/6 WOODLAND PAPER PLATES,12960,1296.00
127166,Denmark,SET/6 STRAWBERRY PAPER CUPS,12960,1296.00
127169,Denmark,SET/6 WOODLAND PAPER CUPS,12744,1274.40
502122,United Kingdom,ASSTD DESIGN 3D PAPER STICKERS,12540,0.00
127167,Denmark,SET/6 STRAWBERRY PAPER PLATES,12480,1248.00
135028,United Kingdom,PACK OF 12 SUKI TISSUES,10000,2500.00
135027,United Kingdom,PACK OF 12 PINK PAISLEY TISSUES,10000,2500.00
135029,United Kingdom,PACK OF 12 RED SPOTTY TISSUES,10000,2500.00
135030,United Kingdom,PACK OF 12 WOODLAND TISSUES,10000,2500.00


## Further Analysis:

Things that I hope to determine in this dataset include: biggest spenders, who is buying the most expensive products, and who has the biggest orders.

In [254]:
df.groupby(by = ["customer_id"]).sum(["total_cost"]).sort_values(by = "total_cost", ascending= False)

,quantity,price,total_cost
customer_id,,,
18102.0,187628,4145.52,606243.25
14646.0,365596,9208.11,523202.74
14156.0,163907,15635.27,299926.26
14911.0,143740,47532.50,270203.05
17450.0,81656,1445.96,235832.75
...,...,...,...
15767.0,-50,50.95,-242.70
16252.0,-158,201.30,-295.09
14337.0,-393,95.88,-658.63


In [255]:
df[df["customer_id"]== 18102.0].groupby(by = ["description"]).sum(["total_cost"]).sort_values(by = "total_cost", ascending= False)

,quantity,price,customer_id,total_cost
description,,,,
VINTAGE UNION JACK MEMOBOARD,5168,114.86,325836.0,32975.92
WOOD BLACK BOARD ANT WHITE FINISH,6001,137.24,524958.0,28301.28
CREAM HEART CARD HOLDER,11324,85.78,651672.0,26897.42
BLACK HEART CARD HOLDER,9938,74.93,579264.0,23182.84
DOORMAT HEARTS,2460,49.22,199122.0,11127.60
...,...,...,...,...
DINOSAURS WRITING SET,56,1.98,36204.0,55.44
GUMBALL COAT RACK,36,1.18,18102.0,42.48
CHILDS APRON SPACEBOY DESIGN,30,1.20,18102.0,36.00


## The client with ID 18102 is the biggest spender, and they have spent about 606,243 total. The products that they spend the most money on are memoboards, blackboards, card holders, and doormats

In [256]:
df.sort_values(by = ["price"],ascending= False).head(15)

,invoice,stock_code,description,quantity,invoice_date,price,customer_id,country,year,total_cost
222671,556444,22502,PICNIC BASKET WICKER 60 PIECES,60,2011-06-10,649.50,15098.0,United Kingdom,2010-2011,38970.00
222682,556446,22502,PICNIC BASKET WICKER 60 PIECES,1,2011-06-10,649.50,15098.0,United Kingdom,2010-2011,649.50
249673,513566,ADJUST2,ADJUSTMENT BY PETER ON JUN 25 2010,1,2010-06-25,358.47,17364.0,United Kingdom,2009-2010,358.47
249672,513564,ADJUST2,ADJUSTMENT BY PETER ON JUN 25 2010,1,2010-06-25,300.13,17050.0,United Kingdom,2009-2010,300.13
268525,515349,22656,VINTAGE BLUE KITCHEN CABINET,1,2010-07-12,295.00,15513.0,United Kingdom,2009-2010,295.00
313339,519887,22655,VINTAGE RED KITCHEN CABINET,1,2010-08-22,295.00,18260.0,United Kingdom,2009-2010,295.00
328960,521482,22656,VINTAGE BLUE KITCHEN CABINET,1,2010-09-06,295.00,15809.0,United Kingdom,2009-2010,295.00
51636,540647,22655,VINTAGE RED KITCHEN CABINET,1,2011-01-10,295.00,17406.0,United Kingdom,2010-2011,295.00
403382,528051,22656,VINTAGE BLUE KITCHEN CABINET,1,2010-10-20,295.00,15874.0,United Kingdom,2009-2010,295.00
282682,516913,22656,VINTAGE BLUE KITCHEN CABINET,1,2010-07-25,295.00,13875.0,United Kingdom,2009-2010,295.00


## The most expensive item that is being ordered overall is the picnic basket wicker 60 pieces item, with a cost of 649.5

In [257]:
df.groupby(by = ["customer_id", "description"]).sum("quantity").sort_values(by = "quantity", ascending = False)

,,quantity,price,total_cost
customer_id,description,,,
13902.0,BLACK AND WHITE PAISLEY FLOWER MUG,25164,0.20,2516.40
15838.0,BROCADE RING PURSE,23760,0.95,4514.40
16754.0,WORLD WAR 2 GLIDERS ASSTD DESIGNS,23040,1.98,4147.20
16422.0,BROCADE RING PURSE,19437,178.47,3834.40
17940.0,PACK OF 12 SUKI TISSUES,17960,1.87,5857.92
...,...,...,...,...
16754.0,LARGE RED RETROSPOT WINDMILL,-720,1.85,-1332.00
15749.0,TEA TIME PARTY BUNTING,-1300,2.55,-3315.00
14277.0,BLUE POLKADOT PUDDING BOWL,-2496,0.13,-324.48


## The highest volume by quantity product is the black and white paisly flower mug, which is ordered by client 13902, although they have only spent 2516.4 total on the product

# Conclusions  

After exploring your dataset, provide a short summary of what you noticed from this dataset.  What would you explore further with more time?

## Conclusions that can be made:

### * The Netherlands has the highest average cost for their orders at 108.754665
### * The United Kingdom has the largest order at 38970.00
### * The client with ID 18102 is the biggest spender, and they have spent about 606,243 total
### * The biggest spending client is ordering: memoboards, blackboards, card holders, and doormats
### * The highest cost product being sold is the picnic basket wicker 60 pieces item, with a cost of 649.5
### * The highest volume by quantity product is the black and white paisly flower mug

## Further Exploration

If I were to explore this data further, I would focus more heavily on visualizations of the data and looking at how the data changes over time. High volume months could be determined and the daily/monthly sales data could be turned into a timeseries. This time series could then be forecast to predict sales in the future